This is a module that segment images. The input is source images and the output is segmented images.

Run the pretrained object segmentation model.

In [1]:
def generate_segmentation(idx):
  # model = pspnet_50_ADE_20K() # load the pretrained model trained on ADE20k dataset
  model = pspnet_101_cityscapes() # load the pretrained model trained on Cityscapes dataset
  # model = pspnet_101_voc12() # load the pretrained model trained on Pascal VOC 2012 dataset 
  # load any of the 3 pretrained models
  out = obj_seg_model.predict_segmentation(
      inp='/content/drive/Shareddrives/Bionic_Eye_IoT_Data/Input/' + str(idx) + '_input.jpeg',
      out_fname='/content/drive/Shareddrives/Bionic_Eye_IoT_Data/Output/' + str(idx) + '_output.jpeg'
  )

Get intermedieate result as a dictionary of electordes that need to be stimulate.
We ended up deciding not to use this intermediate result. Sending segmented images back to client will be sufficient. Client's device should be able to further process the segmented images.

In [ ]:
def get_electrode_name(l):
  names = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y',25:'Z',26:'AA',27:'AB',28:'AC',29:'AD',30:'AE',31:'AF',32:'AG',33:'AH',34:'AI',35:'AJ',36:'AK',37:'AL',38:'AM',39:'AN'}
  d = {}
  for e in l:
    d[names[e[0]]+str(e[1]+1)] = 1
  return d  
  
def get_stimulated_electrodes(idx):
  img = cv2.imread('/content/drive/Shareddrives/Bionic_Eye_IoT_Data/Output/' + str(idx) + '_output.jpeg')
  # plt.imshow(img)

  length = len(img[0])
  height = len(img)
  # print(length,height)
  # from collections import defaultdict

  # key = sumRGB, val = list of pixel tuples [(1,1),(2,2)]
  pixel_vals = defaultdict(list)

  for i in range(0,height,height//39):
    for j in range(0,length,length//39):
      key = sum(img[i][j])
      val = (i//(height//39),j//(length//39))
      pixel_vals[key].append(val)

  # print(pixel_vals)

  # check the color that most electrodes are mapped to
  color_idx = -1
  most_electrodes = 0
  for k,v in pixel_vals.items():
    if len(v)>most_electrodes:
      most_electrodes = len(v)
      color_idx = k
  #print(color_idx)
  #print(len(pixel_vals[color_idx]))
  mask_1 = get_electrode_name(pixel_vals[color_idx])
  return mask_1

For testing purpose, generate percept.

In [ ]:
def generate_percept(stimulated_electrodes):

  implant = p2p.implants.AlphaAMS()
  implant.stim = stimulated_electrodes
  # implant.plot()

  # Set up the model and build it:
  model = p2p.models.ScoreboardModel()
  model.build()

  # Predict the percept and plot it:
  percept = model.predict_percept(implant)
  # percept.plot()
  percept.save('/content/drive/Shareddrives/Bionic_Eye_IoT_Data/Output/' + str(idx) + '_percept.jpeg')

The run API is required by the interface for all modules. This is the method the server runs to start this module.

In [ ]:
def run(num_imgs):
  # f, axarr = plt.subplots(num_imgs, 3) 
  # all_imgs_stimulated_electrodes = []

  for idx in range (1, num_imgs + 1):
    generate_segmentation(idx)
    # stim = get_stimulated_electrodes(idx)
    # all_imgs_stimulated_electrodes.append(stim)

    # for testing purpose
    # generate_percept(mask)
    # axarr[0].imshow(cv2.imread('/content/drive/Shareddrives/Bionic_Eye_IoT_Data/' + str(idx) + '_input.jpeg'))
    # axarr[1].imshow(cv2.imread('/content/drive/Shareddrives/Bionic_Eye_IoT_Data/' + str(idx) + '_output.jpeg'))
    # axarr[2].imshow(cv2.imread('/content/drive/Shareddrives/Bionic_Eye_IoT_Data/' + str(idx) + '_percept.jpeg'))
  # for ele in all_imgs_stimulated_electrodes:
  #   print(ele)

  print("obj_seg_module finished!")

Driver code for this module.

In [ ]:
num_imgs = len(os.listdir('/content/drive/Shareddrives/Bionic_Eye_IoT_Data/Input')) - 1
run(num_imgs)